In [1]:
import pandas as pd

data_112 = pd.read_csv(r"C:\資料探勘\原始資料\DM112.csv")

C:\Users\ruizhong\AppData\Local\Temp\ipykernel_32712\2097710589.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_112 = pd.read_csv(r"C:\資料探勘\原始資料\DM112.csv")


In [11]:
import pandas as pd
data_112 = pd.read_excel(r"C:\資料探勘\經緯度資料標記\112_unlabel.xlsx") 


In [15]:
lat_lon_label_data_112 = pd.read_excel(r'C:\資料探勘\經緯度資料標記\112_label_.xlsx')

In [33]:
missing_lat_lon = lat_lon_label_data_112[lat_lon_label_data_112[['經度','緯度']].isna().any(axis=1)]

In [42]:
lat_lon_label_data_112 = lat_lon_label_data_112[-lat_lon_label_data_112[['經度','緯度']].isna().any(axis=1)]

In [14]:
def processing_address(row):
    
    if "臺北市" not in row["土地位置建物門牌"]:  
        
        return "臺北市" + row["鄉鎮市區"] + row["土地位置建物門牌"] 
    
    else:
        
        return  row["土地位置建物門牌"]

    

In [20]:
print(data_112["土地位置建物門牌"])

0               溪洲段二小段385地號
1            臺北市士林區大西路７４巷２號
2               溪洲段二小段357地號
3               富安段三小段323地號
4             溪洲段三小段137-2地號
                ...        
23024      臺北市萬華區民和街９６號八樓之２
23025      臺北市萬華區民和街９８號四樓之３
23026      臺北市萬華區民和街９８號九樓之３
23027    臺北市萬華區民和街１００號十四樓之２
23028      臺北市萬華區民和街９８號八樓之３
Name: 土地位置建物門牌, Length: 23029, dtype: object


In [ ]:
extract_lat_lon_112 = data_112.copy()
extract_lat_lon_112['土地位置建物門牌'] =  data_112[["鄉鎮市區","土地位置建物門牌"]].apply(processing_address,axis=1)

remove the another purpose for land 

In [16]:
import numpy as np



def delete_interupt_address(string):
    
    try:
    
        if string[-2:] == "地號":

            return "unnormal"

        else :

            return "normal"
        
    
    except Exception as e:
        
        return "unnormal"

    
    


In [ ]:
import numpy as np
extract_lat_lon_112["土地位置建物門牌"]= extract_lat_lon_112["土地位置建物門牌"].apply(lambda x:x if delete_interupt_address(x) == "normal" else np.nan)

In [ ]:
extract_lat_lon_112= extract_lat_lon_112.dropna(subset=["土地位置建物門牌"])



In [7]:
extract_lat_lon_112["土地位置建物門牌"]

1                臺北市士林區大西路７４巷２號
5        臺北市士林區延平北路八段２巷５５弄２０號二樓
6                臺北市士林區中山北路七段９號
7              臺北市士林區中山北路七段９號二樓
8            臺北市士林區士東路２００巷６７之１號
                  ...          
23024          臺北市萬華區民和街９６號八樓之２
23025          臺北市萬華區民和街９８號四樓之３
23026          臺北市萬華區民和街９８號九樓之３
23027        臺北市萬華區民和街１００號十四樓之２
23028          臺北市萬華區民和街９８號八樓之３
Name: 土地位置建物門牌, Length: 19986, dtype: object

In [8]:
extract_lat_lon_112.to_csv("C:\資料探勘\資料探勘作業\extract_lat_lon_112.csv")

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\ruizhong\AppData\Local\Temp\ipykernel_19052\3717339276.py:1: SyntaxWarning: invalid escape sequence '\e'
  extract_lat_lon_112.to_csv("C:\資料探勘\資料探勘作業\extract_lat_lon_112.csv")


In [35]:
import time
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



options = Options()
options.chrome_executable_path = r"C:\資料探勘\chromedriver.exe"
driver = webdriver.Chrome(options=options)
driver.get("https://map.tgos.tw/TGOSCloudMap")


close_button = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "bootbox-close-button.close.btn-close"))
        )
close_button.click()

address_List = []

n = 0
for idx,address in enumerate(missing_lat_lon["土地位置建物門牌"].to_list()): 
    
    try:
    
        # Find and clear the search input field (timeout: 2 seconds)
        search = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "form-control.border-end-0.border-start-0"))
        )

        search.clear()

        # Input the address
        search.send_keys(address)

        # Wait for the search button to be clickable and click it (timeout: 2 seconds)
        search_button = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "fa-solid.fa-magnifying-glass.search"))
        )

        search_button.click()

        time.sleep(random.uniform(5, 10))

        # Wait for the <li> element to be present (timeout: 2 seconds)
        li_element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="map2DArea"]/div[4]/div/div/div[2]/ul/li')))


        # Extract latitude and longitude
        latitude = li_element.get_attribute("data-lat")
        longitude = li_element.get_attribute("data-lng")
        

        
        # Store the latitude and longitude in the corresponding row of the DataFrame
        # Store in the specific row using the index
        
        
        address_List.append([idx,latitude,longitude])
        
        data_112.at[idx,'經度'] = latitude
        data_112.at[idx,'緯度'] = longitude
        
        
        n += 1
        print(f"Processed address {address}: Latitude = {latitude}, Longitude = {longitude},the number of data {n} ")
        
    except Exception as e:
        
        
        try:
                
            print(f"Timeout waiting for result for address {address}. Skipping...")

            data_112.at[idx, '經度'] = pd.NA
            data_112.at[idx, '緯度'] = pd.NA

            address_List.append([idx, None, None])

            time.sleep(120) 
            close_button = WebDriverWait(driver, 300).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "btn.btn-danger.bootbox-accept"))
            )
            close_button.click()
            time.sleep(10)
            
            continue
            
        
        except Exception as e:
            
            print("can't find any element in the search. Skipping ...")
            
            data_112.at[idx, '經度'] = pd.NA
            data_112.at[idx, '緯度'] = pd.NA
            
            
            search = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "form-control.border-end-0.border-start-0"))
            )

            search.clear()
            
            continue
            
   
            
            

Timeout waiting for result for address 臺北市士林區中山北路七段４３巷２號四樓. Skipping...
Processed address 臺北市大同區寧夏路５號: Latitude = 25.054411, Longitude = 121.51492,the number of data 1 
Processed address 臺北市大安區信義路二段１９８巷７之３號: Latitude = 25.032665, Longitude = 121.530682,the number of data 2 
Processed address 臺北市大安區羅斯福路三段２８３巷２９弄３號三樓: Latitude = 25.020575, Longitude = 121.531603,the number of data 3 
Processed address 臺北市大安區光復南路２００巷１１號四樓: Latitude = 25.041907, Longitude = 121.555677,the number of data 4 
Processed address 臺北市大安區信義路二段８６巷２７號四樓: Latitude = 25.033915, Longitude = 121.525409,the number of data 5 
Processed address 臺北市大安區和平東路一段１１５、１１７號房屋地下三層: Latitude = 25.02635, Longitude = 121.533959,the number of data 6 
Processed address 臺北市中山區長春路８８號三樓之１２: Latitude = 25.0548, Longitude = 121.526929,the number of data 7 
Timeout waiting for result for address 臺北市中山區新生北路二段６０之１號，６０之２號，６０之３號，６０之５號，６０之６號，６０之７號，新生北路二段６０巷１號，３號，５號，７號，９號，１１號，１３號，１５號，１７號，１９號，２１號，２３號，２５號，２７號，２９號，新生北路二段６２巷２號，６號，８號，１６號，１８號，２０號，２２號，長春路６

In [36]:
data_112= pd.DataFrame(data_112)

In [44]:
data_112.to_excel(r"C:\資料探勘\經緯度資料標記\112_unlabel.xlsx")

In [45]:
lat_lon_label_data_112.to_excel(r"C:\資料探勘\經緯度資料標記\112_label_.xlsx")

In [37]:
pd.DataFrame({"地址":extract_lat_lon_112["土地位置建物門牌"]}).to_csv("C:\\資料探勘\\資料探勘作業\\address.csv")

NameError: name 'extract_lat_lon_112' is not defined